In [20]:
import pickle
from configs import project_config
import numpy as np
import torch
import os
from tqdm import tqdm
from src.classes.ClassConditionalStbleTgtMarkovianPostMeanDiffTrainer import \
    ConditionalStbleTgtMarkovianPostMeanDiffTrainer
from src.generative_modelling.data_processing import train_and_save_recursive_diffusion_model
from src.generative_modelling.models.ClassVPSDEDiffusion import VPSDEDiffusion
from src.generative_modelling.models.TimeDependentScoreNetworks.ClassConditionalMarkovianTSPostMeanScoreMatching import \
    ConditionalMarkovianTSPostMeanScoreMatching
from utils.data_processing import init_experiment, cleanup_experiment
from utils.math_functions import generate_fQuadSin
from utils.resource_logger import ResourceLogger

In [21]:
# Data parameters
from configs.RecursiveVPSDE.Markovian_fQuadSinHF.recursive_Markovian_PostMeanScore_fQuadSinHF2_LowFTh_T256_H05_tl_110data_StbleTgt import get_config
config = get_config()

In [23]:
assert (config.hurst == 0.5)
assert (config.early_stop_idx == 0)
assert (config.tdata_mult == 110)
#assert (config.sin_space_scale == 25.)
assert (config.feat_thresh == 1./500.)
print(config.scoreNet_trained_path, config.dataSize)
rng = np.random.default_rng()
scoreModel = ConditionalMarkovianTSPostMeanScoreMatching(
    *config.model_parameters)
diffusion = VPSDEDiffusion(beta_max=config.beta_max, beta_min=config.beta_min)


/Users/marcos/GitHubRepos/FractionalBrownianMotion/src/generative_modelling/trained_models/trained_rec_ST_0002FTh_PM_MLP_2LFac_fBiPot_VPSDE_T256_Ndiff10000_Tdiff1000e+00_DiffEmbSz64_ResLay10_ResChan8_DiffHdnSz64_TrueHybd_TrueWghts_t00_dT3906e-03_025a_-05b_00c_MLP_H4_CUp20_tl110 40000


In [27]:
model_dir = "/".join(config.scoreNet_trained_path.split("/")[:-1]) + "/"
entered = False
for file in os.listdir(model_dir):
    if config.scoreNet_trained_path in os.path.join(model_dir, file) and "EE" in file:
        print(file)
        entered = True
        scoreModel.load_state_dict(torch.load(os.path.join(model_dir, file)))
assert entered

AssertionError: 

In [21]:
def experiment_MLP_1D_drifts(config, Xs, PM, onlyGauss=False):
    print("Beta Min : ", config.beta_min)
    if config.has_cuda:
        device = int(os.environ["LOCAL_RANK"])
    else:
        print("Using CPU\n")
        device = torch.device("cpu")
    PM = PM.to(device)
    diffusion = VPSDEDiffusion(beta_max=config.beta_max, beta_min=config.beta_min)
    ts_step = config.deltaT
    print(config.scoreNet_trained_path)
    Xshape = config.ts_length
    num_taus = 100

    num_diff_times = config.max_diff_steps
    Ndiff_discretisation = config.max_diff_steps
    diffusion_times = torch.linspace(start=config.sample_eps, end=config.end_diff_time,
                                     steps=Ndiff_discretisation).to(device)

    features_tensor = torch.stack([Xs for _ in range(1)], dim=0).reshape(Xshape * 1, 1, -1).to(device)
    final_vec_mu_hats = np.zeros(
        (Xshape, num_diff_times, num_taus, config.ts_dims))  # Xvalues, DiffTimes, Ztaus, Ts_Dims
    vec_Z_taus = diffusion.prior_sampling(shape=(Xshape * num_taus, 1, config.ts_dims)).to(device)

    # ts = []
    es = num_diff_times
    ts = []
    # mu_hats_mean = np.zeros((tot_num_feats, num_taus))
    # mu_hats_std = np.zeros((tot_num_feats, num_taus))
    difftime_idx = num_diff_times - 1

    PM.eval()
    for difftime_idx in tqdm(np.arange(num_diff_times - 1, num_diff_times - es - 1, -1)): #difftime_idx >= num_diff_times - es:
        d = diffusion_times[Ndiff_discretisation - (num_diff_times - 1 - difftime_idx) - 1].to(device)
        diff_times = torch.stack([d for _ in range(Xshape)]).reshape(Xshape * 1).to(device)
        eff_times = diffusion.get_eff_times(diff_times=diff_times).unsqueeze(-1).unsqueeze(-1).to(device)
        vec_diff_times = torch.stack([diff_times for _ in range(num_taus)], dim=0).reshape(num_taus * Xshape)
        vec_eff_times = torch.stack([eff_times for _ in range(num_taus)], dim=0).reshape(num_taus * Xshape, 1, 1)
        vec_conditioner = torch.stack([features_tensor for _ in range(num_taus)], dim=0).reshape(
            num_taus * Xshape,
            1, -1)
        with torch.no_grad():
            if onlyGauss:
                scoreEval_vec_Z_taus = diffusion.prior_sampling(shape=vec_Z_taus.shape).to(device)
            else:
                scoreEval_vec_Z_taus = vec_Z_taus
            vec_predicted_score = PM.forward(inputs=scoreEval_vec_Z_taus, times=vec_diff_times, conditioner=vec_conditioner,
                                             eff_times=vec_eff_times)
        vec_scores, vec_drift, vec_diffParam = diffusion.get_conditional_reverse_diffusion(x=vec_Z_taus,
                                                                                           predicted_score=vec_predicted_score,
                                                                                           diff_index=torch.Tensor(
                                                                                               [int((
                                                                                                       num_diff_times - 1 - difftime_idx))]).to(
                                                                                               device),
                                                                                           max_diff_steps=Ndiff_discretisation)
        # assert np.allclose((scores- predicted_score).detach(), 0)
        beta_taus = torch.exp(-0.5 * eff_times[0, 0, 0]).to(device)
        sigma_taus = torch.pow(1. - torch.pow(beta_taus, 2), 0.5).to(device)
        final_mu_hats = (vec_Z_taus / (ts_step * beta_taus)) + ((
                                                                        (torch.pow(sigma_taus, 2) + (
                                                                                torch.pow(beta_taus * config.diffusion,
                                                                                          2) * ts_step)) / (
                                                                                ts_step * beta_taus)) * vec_scores)

        assert (final_mu_hats.shape == (num_taus * Xshape, 1, config.ts_dims))

        means = final_mu_hats.reshape((num_taus, Xshape, config.ts_dims))
        # print(vec_Z_taus.shape, vec_scores.shape)
        final_vec_mu_hats[:, difftime_idx, :] = means.permute((1, 0, 2)).cpu().numpy()
        vec_z = torch.randn_like(vec_drift).to(device)
        vec_Z_taus = vec_drift + vec_diffParam * vec_z
        difftime_idx -= 1
    assert (final_vec_mu_hats.shape == (Xshape, num_diff_times, num_taus, config.ts_dims))
    return final_vec_mu_hats[:, -es:, :, :]

In [22]:
Xshape = 256
Xs = torch.linspace(-1.5, 1.5, steps=Xshape)
true_drifts = (-2. * config.quad_coeff * Xs + config.sin_coeff * config.sin_space_scale * np.sin(config.sin_space_scale * Xs))
final_vec_mu_hats = experiment_MLP_1D_drifts(PM=scoreModel, Xs=Xs, config=config, onlyGauss=True)

Beta Min :  0.0
Using CPU

/Users/marcos/GitHubRepos/FractionalBrownianMotion/src/generative_modelling/trained_models/trained_rec_ST_0002FTh_PM_MLP_2LFac_NFMReg_fQuadSinHF_VPSDE_T256_Ndiff10000_Tdiff1000e+00_DiffEmbSz64_ResLay10_ResChan8_DiffHdnSz64_TrueHybd_TrueWghts_t00_dT3906e-03_05a_004b_250c_MLP_H4_CUp20_tl110


 69%|██████▊   | 6868/10000 [25:20<11:33,  4.52it/s]  

KeyboardInterrupt



In [5]:
remote_file_path = project_config.ROOT_DIR+"data/" #/Users/marcos/Library/CloudStorage/OneDrive-ImperialCollegeLondon/StatML_CDT/Year2/DiffusionModels/ExperimentResults/"
remote_file_path

'/Users/marcos/GitHubRepos/FractionalBrownianMotion/data/'

In [ ]:
np.save(remote_file_path + "QuadSinHF_optimal_tau_experiment_DGauss.npy", final_vec_mu_hats)

In [ ]:
DGauss = np.load(remote_file_path + "QuadSinHF_optimal_tau_experiment_DGauss.npy", allow_pickle=True)
DNonGauss = np.load(remote_file_path + "QuadSinHF_optimal_tau_experiment_DNonGauss.npy", allow_pickle=True)
Xshape = 256
Xs = np.linspace(-1.5, 1.5, num=Xshape)
true_drifts = (-2. * config.quad_coeff * Xs + config.sin_coeff * config.sin_space_scale * np.sin(config.sin_space_scale * Xs))[:, np.newaxis, np.newaxis, np.newaxis]
Gauss_errors = np.mean(DGauss, axis=2) - true_drifts
assert np.all(Gauss_errors.shape == (Xshape, DGauss.shape[1], config.ts_dims))
NonGauss_errors = np.mean(DNonGauss, axis=2) - true_drifts
assert np.all(NonGauss_errors.shape == (Xshape, DNonGauss.shape[1], config.ts_dims))
Gauss_errors = np.sum(np.power(Gauss_errors,2), axis=-1)
NonGauss_errors = np.sum(np.power(NonGauss_errors,2), axis=-1)